<a href="https://colab.research.google.com/github/Speedbird45Bravo/rando_projects/blob/main/Pipe_61521.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches_latest.csv").dropna()
df = df[(df.league=="Barclays Premier League") | (df.league=="UEFA Champions League")].reset_index(drop=True)

In [3]:
df['spi_margin'] = df['spi1'] - df['spi2']
df['goal_margin'] = df['score1'] - df['score2']

In [4]:
spis = []
results = []

In [5]:
for x in df['spi_margin']:
  if x > 0:
    spis.append("HOME")
  else:
    spis.append("AWAY")

In [6]:
for y in df['goal_margin']:
  if y > 0:
    results.append("HOME")
  elif y < 0:
    results.append("AWAY")
  else:
    results.append("DRAW")

In [7]:
df['spi_margin'] = spis
df['goal_margin'] = results

In [8]:
matches = []

In [9]:
for z in range(len(df)):
  z = df.iloc[z,:]
  if z['goal_margin'] == z['spi_margin']:
    matches.append("MATCH")
  else:
    matches.append("NO MATCH")

In [10]:
df['match'] = matches

In [11]:
X_final = df[['spi1','spi2','score1','score2']]
y_final = df['match']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=3)

In [13]:
pipe = Pipeline([('Feature Selector', VarianceThreshold()), ('Scaler', StandardScaler()), ('Classifier', XGBClassifier(base_score=0.56, max_depth=6, n_estimators=400))])

In [15]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('Feature Selector', VarianceThreshold(threshold=0.0)),
                ('Scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('Classifier',
                 XGBClassifier(base_score=0.56, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=None,
                               n_estimators=400, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)

In [16]:
def r_3_score(x, y):
  score = pipe.score(x, y)
  num = score * 100
  return num

In [17]:
pipe_train = r_3_score(X_train, y_train).round(3)
pipe_test = r_3_score(X_test, y_test).round(3)

In [19]:
print("Train Score: {}% | Test Score: {}%".format(pipe_train, pipe_test))

Train Score: 100.0% | Test Score: 92.079%
